# Inferring

- We will infer sentiment and topics from product reviews and news articles.

## Setup

- Load the API key and relevant Python libaries.

In [3]:
from langchain_groq import ChatGroq
import langchain_core
import os
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

groq_api_key = os.getenv('GROQ_API_KEY')

In [5]:
# model='llama-3.3-70b-versatile'

llm = ChatGroq(
    api_key=groq_api_key,
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None
)

In [7]:
def get_completion(prompt, model="llama-3.3-70b-versatile"):
    llm.model_name = model
    return llm.invoke(prompt).content

## Product Review Text

In [10]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [13]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is overwhelmingly positive. The customer is satisfied with the product, the price, and the shipping speed. They also had a few issues with the product (broken string and missing part), but the company's customer support resolved these problems quickly and efficiently, which further reinforced the customer's positive opinion of the company. The use of phrases such as "Lumina seems to me to be a great company" and "cares about their customers and products" emphasizes the customer's enthusiasm and trust in the brand. Overall, the review expresses a strong sense of satisfaction and loyalty towards the company.


In [15]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive


## Identify types of emotions

In [18]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

satisfaction, happiness, appreciation, relief, gratitude


## Identify anger

In [21]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

no


## Extract product and company name from customer reviews

In [24]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "Item": "lamp",
  "Brand": "Lumina"
}
```


## Doing multiple tasks at once

In [27]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}
```


## Inferring topics

In [30]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [33]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

NASA, Employees, Survey, Government, Satisfaction


In [35]:
response.split(sep=',')

['NASA', ' Employees', ' Survey', ' Government', ' Satisfaction']

In [37]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [40]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as follows:
item from the list: 0 or 1

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Here are the results:

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [52]:
topic_dict = {k.strip(): int(v.strip()) for line in response.split("\n")
              if ": " in line and (k := line.split(": ", 1)[0]) and (v := line.split(": ", 1)[1]).strip() in ("0", "1")}


if topic_dict.get("nasa") == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


In [56]:
topic_dict = {}
for line in response.split("\n"):
    if ": " not in line:
        continue
    k, v = line.split(": ", 1)
    if v.strip() in ("0", "1"):
        topic_dict[k.strip()] = int(v.strip())

if topic_dict.get("nasa") == 1:
    print("ALERT: New NASA story!!!")

ALERT: New NASA story!!!
